# API Documentation 
https://www.alphavantage.co/documentation/

In this Notebook, I will utilize the Alpha Vantage API to pull stock data for a given ticker. 
- Pull in Latest News and Sentiment Analysis
  - Summarize using T5 model and tokenizer from TensorFlow


In [ ]:
import requests
import json
from datetime import datetime
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

def summarize_text(text, model, tokenizer, max_length=512):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="tf", max_length=max_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=50, min_length=25, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)

# Path to your API key file
api_key_file_path = r"C:\Users\david\OneDrive\Documents\API Keys\AlphaVantage.txt"

# Read API key from file
with open(api_key_file_path, 'r') as file:
    API = file.read().strip()

ticker = "FSM"
startdate = datetime.strptime("12/01/2023", "%m/%d/%Y")
enddate = datetime.today()
limit = 5

news_and_sentiments_url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&timefrom={startdate.strftime("%Y-%m-%d")}TO{enddate.strftime("%Y-%m-%d")}&limit={limit}&apikey={API}'

try:
    response = requests.get(news_and_sentiments_url)

    if response.status_code == 200:
        news_and_sentiments_data = response.json()

        for item in news_and_sentiments_data.get('feed', []):
            time_published_str = item.get('time_published', '')
            if time_published_str:
                time_published = datetime.strptime(time_published_str, '%Y%m%dT%H%M%S')

                if startdate <= time_published <= enddate:
                    # Filter the ticker_sentiment to only include FSM
                    item['ticker_sentiment'] = [ts for ts in item.get('ticker_sentiment', []) if ts['ticker'] == ticker]

                    # Summarize the news item
                    summary = summarize_text(item.get('summary', ''), model, tokenizer)
                    print("Original Summary:", item.get('summary', ''))
                    print("T5 Summary:", summary)
                    print("\n")
    else:
        print(f"Failed to fetch data: Status code {response.status_code}")
except Exception as e:
    print(f"An error occurred: {e}")
